In [1]:
# Чтобы скачать и распаковать zip-архив в output папку в Питоне:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go 

from openpyxl import Workbook
from IPython.display import display

pd.set_option('display.max_columns', 40) # выводить больше столбцов
pd.set_option('display.max_rows', 40)   # выводить больше строк
plt.style.use('default')    # корректный вывод графиков в темной теме 

# fig.show(renderer='notebook') #включение анимации

#!/usr/bin/env python
import io
import zipfile
import requests  # $ pip install requests



r = requests.get("https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip")
with r, zipfile.ZipFile(io.BytesIO(r.content)) as archive:
    archive.extractall('Data')


# https://lms.skillfactory.ru/assets/courseware/v1/6559ab1e1d17acac79bec5dc8052261b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/diabetes_data.zip

In [2]:

diab = pd.read_csv('data/diabetes_data.csv')
diab.info()
with pd.ExcelWriter('task_befor_85.xlsx') as writer:  
    diab.to_excel(writer, sheet_name='Start_csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 778 entries, 0 to 777
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               778 non-null    int64  
 1   Glucose                   778 non-null    int64  
 2   BloodPressure             778 non-null    int64  
 3   SkinThickness             778 non-null    int64  
 4   Insulin                   778 non-null    int64  
 5   BMI                       778 non-null    float64
 6   DiabetesPedigreeFunction  778 non-null    float64
 7   Age                       778 non-null    int64  
 8   Outcome                   778 non-null    int64  
 9   Gender                    778 non-null    object 
dtypes: float64(2), int64(7), object(1)
memory usage: 60.9+ KB


In [3]:
# fig = px.histogram(diab, x="Pregnancies",  marginal="box", nbins=30, width=1000 , height= 500, color='Outcome')
# fig.show()

In [4]:
diab_81 = diab.drop_duplicates()
diab_81.shape
with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_81.to_excel(writer, sheet_name='drop_dup_81')


In [5]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diab_81.columns:
    #наибольшая относительная частота в признаке
    top_freq = diab_81[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diab_81[col].nunique() / diab_81[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.99:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
print(low_information_cols)

Gender: 100.0% одинаковых значений
['Gender']


In [6]:
diab_82 = diab_81.drop(low_information_cols, axis=1)
diab_82.head()
with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_82.to_excel(writer, sheet_name='drop_gender_82')


In [7]:
# Задание 8.3
# Попробуйте найти пропуски в данных с помощью метода insull().
# Спойлер: ничего не найдёте. А они есть! Просто они скрыты от наших глаз. В таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI
# обозначены нулём, поэтому традиционные методы поиска пропусков ничего вам не покажут. Давайте это исправим!
# Замените все записи, равные 0, в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI на символ пропуска. Его вы можете взять из библиотеки numpy: np.nan.
# Какая доля пропусков содержится в столбце Insulin? Ответ округлите до сотых.

diab_83 = diab_82.copy()
col_to_change = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']

for col in col_to_change:
    diab_83[col] = diab_83[col].replace({0: np.nan})
display(diab_83.head())

round(diab_83['Insulin'].isna().value_counts(normalize=True)[True], 2)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,190.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,NaN,35.7,0.148,21,0
2,2,108.0,64.0,NaN,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,NaN,29.9,0.210,50,0


0.49

In [8]:
# Задание 8.4
# Удалите из данных признаки, где число пропусков составляет более 30 %. Сколько признаков осталось в ваших данных (с учетом удаленных неинформативных признаков в задании 8.2)?
display(diab_83.shape)
n = diab_83.shape[0] #число строк в таблице
thresh_col = n*0.7
diab_84 = diab_83.dropna(how='any', thresh=thresh_col, axis=1)
display(diab_84)
display(diab_84.shape)
with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_84.to_excel(writer, sheet_name='drop_Insulin_84')

(768, 9)

,Pregnancies,Glucose,BloodPressure,SkinThickness,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,98.0,58.0,33.0,34.0,0.430,43,0
1,2,112.0,75.0,32.0,35.7,0.148,21,0
2,2,108.0,64.0,NaN,30.8,0.158,21,0
3,8,107.0,80.0,NaN,24.6,0.856,34,0
4,7,136.0,90.0,NaN,29.9,0.210,50,0
...,...,...,...,...,...,...,...,...
763,5,139.0,64.0,35.0,28.6,0.411,26,0
764,1,96.0,122.0,NaN,22.4,0.207,27,0
765,10,101.0,86.0,37.0,45.6,1.136,38,1
766,0,141.0,NaN,NaN,42.4,0.205,29,1


(768, 8)

In [9]:
# Задание 8.5
# Удалите из данных только те строки, в которых содержится более двух пропусков одновременно. Чему равно результирующее число записей в таблице?

display(diab_84.shape)

m = diab_84.shape[1]
tresh_line = m - 2 # убираем те записи где больее 2х Nan
print(f'triesh_line = {tresh_line}')
diab_85 = diab_84.dropna(how='any', thresh=tresh_line, axis=0)

with pd.ExcelWriter('task_befor_85.xlsx', mode='a') as writer:  
    diab_85.to_excel(writer, sheet_name='drop_lines_85')
display(diab_85.info())

display(diab_85.shape)

# values = {
#     'one': df['one'].mean(),
#     'two': df['two'].mean(),
#     'three': df['three'].mean(),
#     'four': df['four'].mode()[0]
# }


(768, 8)

triesh_line = 6
<class 'pandas.core.frame.DataFrame'>
Int64Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


None

(761, 8)

In [10]:
# Задание 8.6
# # В оставшихся записях замените пропуски на медиану. Чему равно среднее значение в столбце SkinThickness? Ответ округлите до десятых.
print(round(diab_85['SkinThickness'].mean(),1))
diab_86= diab_85.copy()
cols = list(diab_86.columns)
cols.remove('Outcome')
print(col_to_change)
values = {
    'Glucose': diab_86['Glucose'].median(),
    'BloodPressure': diab_86['BloodPressure'].median(),
    'SkinThickness': diab_86['SkinThickness'].median(),
    'BMI': diab_86['BMI'].median()}
#заполняем оставшиеся записи константами в соответствии со словарем values
diab_86 = diab_86.fillna(values)
display(round(diab_86['SkinThickness'].mean(),1))

29.2
['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']


29.1

In [11]:
# Задание 8.7

# Сколько выбросов найдёт классический метод межквартильного размаха в признаке SkinThickness?
def outliers_iqr_mod(data, feature, left =1.5, right= 1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * left)
    upper_bound = quartile_3 + (iqr * right)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(diab_86, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [12]:
def outliers_z_score_mod(data, feature, left = 3, right = 3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score_mod(diab_86, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [13]:
outliers, cleaned = outliers_iqr_mod(diab_86, 'DiabetesPedigreeFunction')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

outliers, cleaned = outliers_iqr_mod(diab_86, 'DiabetesPedigreeFunction', log_scale=True)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


Число выбросов по методу Тьюки: 29
Результирующее число записей: 732
Число выбросов по методу z-отклонения: 0
Результирующее число записей: 761


Имеются две базы данных (два листа Excel-файла): база с ценами конкурентов (Data_Parsing) и внутренняя база компании (Data_Company).

В базе парсинга есть два id, однозначно определяющие товар: producer_id и producer_color.

В базе компании есть два аналогичных поля: item_id и color_id.

Нам известно, что коды в двух базах отличаются наличием набора служебных символов. В базе парсинга встречаются следующие символы: _, -, ~, \\, /.

Необходимо:

Считать данные из Excel в DataFrame (Data_Parsing) и (Data_Company).
Подтянуть к базе парсинга данные из базы компании (item_id, color_id, current_price) и сформировать столбец разницы цен в % (цена конкурента к нашей цене).
Определить сильные отклонения от среднего в разности цен в пределах бренда-категории (то есть убрать случайные выбросы, сильно искажающие сравнение). Критерий — по вкусу, написать комментарий в коде.
Записать новый файл Excel с базой парсинга, приклееными к ней столбцами из пункта 2 и с учётом пункта 3 (можно добавить столбец outlier и проставить Yes для выбросов).



In [14]:
Data_Parsing = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Parsing')
Data_Company = pd.read_excel('data/Data_TSUM.xlsx', sheet_name='Data_Company')
display(Data_Company)
display(Data_Parsing)

,brand,Category,item_id,color_id,current price
0,Valentino,Shoes,aaaaa111111,black,247
1,Valentino,Shoes,aaaaa111112,black,161
2,Valentino,Shoes,aaaaa111113,black,234
3,Valentino,Shoes,aaaaa111114,black,167
4,Valentino,Shoes,aaaaa111115,black,153
...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,165
71,Stone Island,Bags,sssss111132,red,196
72,Stone Island,Bags,sssss111133,red,236
73,Stone Island,Bags,sssss111134,red,222


,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa1111_11,black,167
1,Valentino,Shoes,aaaaa1111_12,black,188
2,Valentino,Shoes,aaaaa1111_13,black,184
3,Valentino,Shoes,aaaaa1111_14,bla//ck,196
4,Valentino,Shoes,aaaaa1111_15,bla\\ck,250
...,...,...,...,...,...
70,Stone Island,Bags,sssss1111_31,~~~red,164
71,Stone Island,Bags,sssss1111_32,~~~red,158
72,Stone Island,Bags,sssss1111_33,~~~red,194
73,Stone Island,Bags,sssss1111_34,~~~red,256


In [15]:
import re
signs_of_signs = ['~','_','-', '/', '\\\\']
Data_Parsing.replace(to_replace=signs_of_signs, value="", regex=True, inplace=True)
Data_Parsing

,brand,Category,producer_id,producer_color,price
0,Valentino,Shoes,aaaaa111111,black,167
1,Valentino,Shoes,aaaaa111112,black,188
2,Valentino,Shoes,aaaaa111113,black,184
3,Valentino,Shoes,aaaaa111114,black,196
4,Valentino,Shoes,aaaaa111115,black,250
...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164
71,Stone Island,Bags,sssss111132,red,158
72,Stone Island,Bags,sssss111133,red,194
73,Stone Island,Bags,sssss111134,red,256


In [16]:
Data_merged = Data_Parsing.merge(Data_Company, how='left', left_on=['producer_id',  'producer_color'], right_on=['item_id', 'color_id'])
Data_merged['Diff_%'] = round((Data_merged['price'] - Data_merged['current price']) / Data_merged['current price']*100 , 2)
Data_merged

,brand_x,Category_x,producer_id,producer_color,price,brand_y,Category_y,item_id,color_id,current price,Diff_%
0,Valentino,Shoes,aaaaa111111,black,167,Valentino,Shoes,aaaaa111111,black,247,-32.39
1,Valentino,Shoes,aaaaa111112,black,188,Valentino,Shoes,aaaaa111112,black,161,16.77
2,Valentino,Shoes,aaaaa111113,black,184,Valentino,Shoes,aaaaa111113,black,234,-21.37
3,Valentino,Shoes,aaaaa111114,black,196,Valentino,Shoes,aaaaa111114,black,167,17.37
4,Valentino,Shoes,aaaaa111115,black,250,Valentino,Shoes,aaaaa111115,black,153,63.40
...,...,...,...,...,...,...,...,...,...,...,...
70,Stone Island,Bags,sssss111131,red,164,Stone Island,Bags,sssss111131,red,165,-0.61
71,Stone Island,Bags,sssss111132,red,158,Stone Island,Bags,sssss111132,red,196,-19.39
72,Stone Island,Bags,sssss111133,red,194,Stone Island,Bags,sssss111133,red,236,-17.80
73,Stone Island,Bags,sssss111134,red,256,Stone Island,Bags,sssss111134,red,222,15.32


In [17]:
fig = px.box(Data_merged, x = 'brand_x', y='Diff_%', color='Category_x')
fig.show()

In [18]:
Data_merged['brand_x'].value_counts().index

Index(['Valentino', 'Brioni', 'Stone Island'], dtype='object')

In [19]:
new_merged = pd.DataFrame()
all_outliers = pd.DataFrame()
for brand in Data_merged['brand_x'].value_counts().index:
    for category in Data_merged['Category_x'].value_counts().index:
        mask1 = Data_merged['brand_x'] == brand
        mask2 =Data_merged['Category_x'] == category
        mask = mask1 & mask2
        outliers, cleaned = outliers_iqr_mod(Data_merged[mask], 'Diff_%')
        print(f'Число выбросов по методу Тьюки:{brand}-{category} = {outliers.shape[0]}')
        print(f'Результирующее число записей:{brand}-{category} =  {cleaned.shape[0]}')
        new_merged = pd.concat([new_merged, cleaned], axis= 0)
        all_outliers = pd.concat([all_outliers, outliers], axis= 0)
fig = px.box(new_merged, x = 'brand_x', y='Diff_%', color='Category_x')
fig.show()


Число выбросов по методу Тьюки:Valentino-Shoes = 0
Результирующее число записей:Valentino-Shoes =  14
Число выбросов по методу Тьюки:Valentino-Bags = 0
Результирующее число записей:Valentino-Bags =  11
Число выбросов по методу Тьюки:Brioni-Shoes = 4
Результирующее число записей:Brioni-Shoes =  10
Число выбросов по методу Тьюки:Brioni-Bags = 0
Результирующее число записей:Brioni-Bags =  11
Число выбросов по методу Тьюки:Stone Island-Shoes = 0
Результирующее число записей:Stone Island-Shoes =  14
Число выбросов по методу Тьюки:Stone Island-Bags = 1
Результирующее число записей:Stone Island-Bags =  10


In [20]:
Data_merged.groupby(['brand_x', 'Category_x'], as_index=False)['Diff_%'].agg(['mean', 'std'])

mean        std
brand_x      Category_x                     
Brioni       Bags        4.051818  29.313723
             Shoes       5.303571  27.620552
Stone Island Bags       -0.906364  26.079932
             Shoes       4.049286  27.633944
Valentino    Bags        2.974545  30.178997
             Shoes       0.088571  35.835243